### Bayesian Logistic Binary

In [94]:
# Bayesian Logistic Regression for Binary Classification using Pyro
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
import itertools
from pyro.infer.autoguide import AutoMultivariateNormal
palette = itertools.cycle(sns.color_palette())

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

In [95]:
# Load the data set
data = pd.read_csv('train_processed.csv')

In [96]:
# Separate between features/inputs (X) and target/output variables (y)
mat = data.values
X = np.delete(mat, 1, axis=1)
print(X.shape)
y = mat[:, 1].astype("int")
print(y.shape)
ind = mat[:,1].astype("int")  #and get the indexes
print(ind.shape)

(1058, 53)
(1058,)
(1058,)


In [97]:
# standardize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

/var/folders/1m/z6bjwn212yg9839rhl52qvtm0000gn/T/ipykernel_7831/822795065.py:4: RuntimeWarning: invalid value encountered in divide
  X = (X - X_mean) / X_std


In [98]:
train_perc = 0.66 # percentage of training data
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y)) # we also randomize the dataset
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
X_test = X[ix_test,:]
y_train = y[ix_train]
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

num train: 698
num test: 360


Logistic regression

In [99]:
from sklearn.impute import SimpleImputer

# Create imputer (mean strategy works well for most numeric data)
imputer = SimpleImputer(strategy='mean')

# Fit to training data and transform both train and test
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

/Users/paulagranlund/Library/Python/3.9/lib/python/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [ 4 15 20]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/paulagranlund/Library/Python/3.9/lib/python/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [ 4 15 20]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [100]:
# create and fit logistic regression model
logreg = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto', C=1)
logreg.fit(X_train, y_train)

# make predictions for test set
y_hat = logreg.predict(X_test)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
true values: [0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0

/Users/paulagranlund/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [80]:
# Model definition
def model(X, n_cat, y=None):
    n_features = X.shape[1]

    beta = pyro.sample("beta", dist.Normal(0., 1.).expand([n_features, 1]).to_event(2))
    logits = X @ beta  # shape: (N, 1)

    with pyro.plate("data", X.shape[0]):
        y = pyro.sample("y", dist.Bernoulli(logits=logits.squeeze(-1)), obs=y)
    
    return y

In [110]:
def model(X, n_cat, y=None):
    n_features = X.shape[1]

    # Sample one weight per feature → beta has shape (D,)
    beta = pyro.sample("beta", dist.Normal(0., 1.).expand([n_features]).to_event(1))

    logits = X @ beta  # shape: (N,)

    with pyro.plate("data", X.shape[0]):
        pyro.sample("y", dist.Bernoulli(logits=logits), obs=y)


In [111]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

/var/folders/1m/z6bjwn212yg9839rhl52qvtm0000gn/T/ipykernel_7831/2804050145.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
/var/folders/1m/z6bjwn212yg9839rhl52qvtm0000gn/T/ipykernel_7831/2804050145.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).float()
/var/folders/1m/z6bjwn212yg9839rhl52qvtm0000gn/T/ipykernel_7831/2804050145.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test).float()
/var/folders/1m/z6bjwn212yg9839rhl52qvtm0000g

In [112]:
# Clear previous state
pyro.clear_param_store()

# Define guide
guide = AutoMultivariateNormal(model)

# Optimizer and ELBO
optimizer = ClippedAdam({"lr": 0.001})
elbo = Trace_ELBO()
svi = SVI(model, guide, optimizer, loss=elbo)

# Training loop
n_steps = 40000
for step in range(n_steps):
    loss = svi.step(X_train, n_cat=None, y=y_train.float())
    if step % 1000 == 0:
        print(f"[Step {step}] ELBO: {loss:.2f}")

[Step 0] ELBO: 914.22
[Step 1000] ELBO: 532.35
[Step 2000] ELBO: 525.32
[Step 3000] ELBO: 516.93
[Step 4000] ELBO: 517.03
[Step 5000] ELBO: 519.89
[Step 6000] ELBO: 516.25
[Step 7000] ELBO: 520.26
[Step 8000] ELBO: 513.12
[Step 9000] ELBO: 515.85
[Step 10000] ELBO: 519.35
[Step 11000] ELBO: 510.90
[Step 12000] ELBO: 517.40
[Step 13000] ELBO: 514.13
[Step 14000] ELBO: 517.32
[Step 15000] ELBO: 512.85
[Step 16000] ELBO: 513.78
[Step 17000] ELBO: 518.28
[Step 18000] ELBO: 517.67
[Step 19000] ELBO: 512.15
[Step 20000] ELBO: 514.59
[Step 21000] ELBO: 514.26
[Step 22000] ELBO: 514.13
[Step 23000] ELBO: 514.24
[Step 24000] ELBO: 518.14
[Step 25000] ELBO: 514.37
[Step 26000] ELBO: 513.16
[Step 27000] ELBO: 512.00
[Step 28000] ELBO: 514.73
[Step 29000] ELBO: 514.31
[Step 30000] ELBO: 512.43
[Step 31000] ELBO: 511.17
[Step 32000] ELBO: 513.46
[Step 33000] ELBO: 517.75
[Step 34000] ELBO: 514.02
[Step 35000] ELBO: 512.86
[Step 36000] ELBO: 517.63
[Step 37000] ELBO: 513.09
[Step 38000] ELBO: 514.17

In [106]:
# Posterior predictive
samples = guide()
beta_samples = samples['beta'].detach()
beta_mean = beta_samples.mean()  # shape (D,)

# Predict probabilities
logits_test = X_test @ beta_mean
probs_test = torch.sigmoid(logits_test)
y_pred = (probs_test > 0.5).int()

# Accuracy
accuracy = (y_pred == y_test).float().mean()
print(f"Test Accuracy: {accuracy:.3f}")

RuntimeError: both arguments to matmul need to be at least 1D, but they are 2D and 0D

In [105]:
print("beta_samples shape:", beta_samples.shape)  # (num_samples, D)
print("beta_mean shape:", beta_mean.shape)        # (D,)
print("X_test shape:", X_test.shape)              # (N, D)

beta_samples shape: torch.Size([50])
beta_mean shape: torch.Size([])
X_test shape: torch.Size([360, 50])


In [21]:
# Visualize posterior distributions of betas
for d in range(beta_samples.shape[0]):
    sns.histplot(beta_samples[d].squeeze(), kde=True, alpha=0.3)
plt.title("Posterior distributions of beta coefficients")
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

RuntimeError: size mismatch, got input (360), mat (360x50), vec (1)